In [ ]:
import pandas as pd
import mysql.connector
import os

# Configuración de la conexión a la base de datos MySQL
try:
    mydb = mysql.connector.connect(
        host="roundhouse.proxy.rlwy.net",
        port=38517,
        user="root",
        password="wZGotyxIDqpAtQjPxNuxxezSbbroztiw",
        database="railway"
    )
except mysql.connector.Error as err:
    print(f"Error de MySQL: {err}")
    exit()
except Exception as e:
    print(f"Error general: {e}")
    exit()

if mydb.is_connected():
    print("Conexión a la base de datos exitosa.")
    mycursor = mydb.cursor()

    # Directorio donde se encuentran los archivos de entrada
    input_dir = r"C:\Users\AXG928866\Documents\U\archivos fuentes\archivos adecuados\input"
    
    # Obtener únicamente archivos con extensión .xlsx
    archivos = [archivo for archivo in os.listdir(input_dir) if archivo.lower().endswith('.xlsx')]

    if not archivos:
        print(f"No se encontraron archivos Excel en la ruta: {input_dir}")
    else:
        total_registros_totales = 0

        for archivo in archivos:
            ruta_excel = os.path.join(input_dir, archivo)
            print(f"\nProcesando archivo: {archivo}")
            
            # Intentar leer el archivo Excel
            try:
                df = pd.read_excel(ruta_excel, header=7)
            except FileNotFoundError:
                print(f"Error: No se encontró el archivo: {ruta_excel}")
                continue
            except pd.errors.ParserError as e:
                print(f"Error al leer el archivo Excel (formato incorrecto) en {archivo}: {e}")
                continue
            except Exception as e:
                print(f"Error general al leer el archivo {archivo}: {e}")
                continue

            # Validar que el archivo tenga al menos 15 columnas para poder extraer los índices requeridos
            if df.shape[1] < 15:
                print(f"El archivo {archivo} no cumple con la cantidad requerida de columnas para ser cargado.")
                continue

            # Intentar seleccionar las columnas específicas y renombrarlas
            try:
                df = df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14]]
                df.columns = [
                    "Codigo_contable", "NombreVehiculo", "Comprobante", "Secuencia", "FechaElaboracion",
                    "IdentificacionTercero", "NombreTercero", "Descripcion", "Debito", "TipoMantenimiento",
                    "Categoria", "Matricula", "TipoMatricula"
                ]
            except Exception as e:
                print(f"Error al procesar las columnas del archivo {archivo}: {e}")
                continue

            registros_insertados = 0

            # Procesar cada fila e insertar en la base de datos
            for index, row in df.iterrows():
                try:
                    fecha_elaboracion = (
                        pd.to_datetime(row['FechaElaboracion'], dayfirst=True).strftime('%Y-%m-%d')
                        if pd.notnull(row['FechaElaboracion']) else None
                    )

                    sql = """
                    INSERT INTO STG_Mantenimientos 
                    (Codigo_contable, NombreVehiculo, Comprobante, Secuencia, FechaElaboracion, 
                     IdentificacionTercero, NombreTercero, Descripcion, Debito, TipoMantenimiento, Categoria, Matricula, TipoMatricula) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """
                    val = (
                        row['Codigo_contable'], row['NombreVehiculo'], row['Comprobante'], row['Secuencia'],
                        fecha_elaboracion, row['IdentificacionTercero'], row['NombreTercero'], row['Descripcion'],
                        row['Debito'], row['TipoMantenimiento'], row['Categoria'], row['Matricula'], row['TipoMatricula']
                    )
                    mycursor.execute(sql, val)
                    mydb.commit()
                    registros_insertados += mycursor.rowcount
                except mysql.connector.IntegrityError as e:
                    if "Duplicate entry" in str(e):
                        print(f"Advertencia en {archivo}: Registro duplicado omitido: {row['Codigo_contable']}, {row['Comprobante']}, {row['Secuencia']}")
                    else:
                        print(f"Error al insertar registro en {archivo}: {e}")
                        mydb.rollback()
                except Exception as e:
                    print(f"Error general durante la inserción en {archivo}: {e}")
                    mydb.rollback()

            print(f"Archivo {archivo}: {registros_insertados} registros insertados.")
            total_registros_totales += registros_insertados

        print(f"\nTotal de registros insertados de todos los archivos: {total_registros_totales}")

    mycursor.close()
    mydb.close()
    print("Conexión a la base de datos cerrada.")
else:
    print("No se pudo conectar a la base de datos.")


In [ ]:
import pandas as pd
import mysql.connector
import os

# Configuración de la conexión a la base de datos MySQL
try:
    mydb = mysql.connector.connect(
        host="roundhouse.proxy.rlwy.net",
        port=38517,
        user="root",
        password="wZGotyxIDqpAtQjPxNuxxezSbbroztiw",
        database="railway"
    )
except mysql.connector.Error as err:
    print(f"Error de MySQL: {err}")
    exit()
except Exception as e:
    print(f"Error general: {e}")
    exit()

if mydb.is_connected():
    print("Conexión a la base de datos exitosa.")
    mycursor = mydb.cursor()

    # Borrar todos los datos de la tabla antes de la inserción
    try:
        mycursor.execute("DELETE FROM STG_Mantenimientos")
        mydb.commit()
        print("Datos anteriores eliminados de la tabla STG_Mantenimientos.")
    except mysql.connector.Error as err:
        print(f"Error al borrar datos de la tabla: {err}")
        mycursor.close()
        mydb.close()
        exit()
    
    # Directorio de entrada
    input_dir = r"C:\Users\AXG928866\Documents\U\archivos fuentes\archivos adecuados\input"
    archivos = [archivo for archivo in os.listdir(input_dir) if archivo.lower().endswith('.xlsx')]

    if not archivos:
        print(f"No se encontraron archivos Excel en la ruta: {input_dir}")
        mycursor.close()
        mydb.close()
        exit()

    # Diccionario para almacenar el resumen de cada archivo
    resumen_archivos = {}

    for archivo in archivos:
        resumen_archivos[archivo] = {"cumple": False, "registros": 0, "mensaje": ""}
        ruta_excel = os.path.join(input_dir, archivo)
        print(f"\nProcesando archivo: {archivo}")

        # Intentar leer el archivo Excel
        try:
            df = pd.read_excel(ruta_excel, header=7)
        except Exception as e:
            mensaje = f"Error al leer el archivo: {e}"
            print(mensaje)
            resumen_archivos[archivo]["mensaje"] = mensaje
            continue

        # Validar cantidad de columnas requerida (al menos 15)
        if df.shape[1] < 15:
            mensaje = "No cumple con la cantidad requerida de columnas."
            print(f"El archivo {archivo} {mensaje}")
            resumen_archivos[archivo]["mensaje"] = mensaje
            continue

        # Intentar seleccionar y renombrar las columnas requeridas
        try:
            df = df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14]]
            df.columns = [
                "Codigo_contable", "NombreVehiculo", "Comprobante", "Secuencia", "FechaElaboracion",
                "IdentificacionTercero", "NombreTercero", "Descripcion", "Debito", "TipoMantenimiento",
                "Categoria", "Matricula", "TipoMatricula"
            ]
        except Exception as e:
            mensaje = f"Error al procesar las columnas: {e}"
            print(mensaje)
            resumen_archivos[archivo]["mensaje"] = mensaje
            continue

        resumen_archivos[archivo]["cumple"] = True
        registros_insertados = 0

        # Insertar registros en la base de datos
        for index, row in df.iterrows():
            try:
                fecha_elaboracion = (
                    pd.to_datetime(row['FechaElaboracion'], dayfirst=True).strftime('%Y-%m-%d')
                    if pd.notnull(row['FechaElaboracion']) else None
                )

                sql = """
                INSERT INTO STG_Mantenimientos 
                (Codigo_contable, NombreVehiculo, Comprobante, Secuencia, FechaElaboracion, 
                 IdentificacionTercero, NombreTercero, Descripcion, Debito, TipoMantenimiento, Categoria, Matricula, TipoMatricula) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                val = (
                    row['Codigo_contable'], row['NombreVehiculo'], row['Comprobante'], row['Secuencia'],
                    fecha_elaboracion, row['IdentificacionTercero'], row['NombreTercero'], row['Descripcion'],
                    row['Debito'], row['TipoMantenimiento'], row['Categoria'], row['Matricula'], row['TipoMatricula']
                )
                mycursor.execute(sql, val)
                mydb.commit()
                registros_insertados += mycursor.rowcount
            except mysql.connector.IntegrityError as e:
                if "Duplicate entry" in str(e):
                    print(f"Advertencia en {archivo}: Registro duplicado omitido: {row['Codigo_contable']}, {row['Comprobante']}, {row['Secuencia']}")
                else:
                    print(f"Error al insertar registro en {archivo}: {e}")
                    mydb.rollback()
            except Exception as e:
                print(f"Error general durante la inserción en {archivo}: {e}")
                mydb.rollback()

        resumen_archivos[archivo]["registros"] = registros_insertados
        print(f"Archivo {archivo}: {registros_insertados} registros insertados.")

    # Mostrar resumen final de todos los archivos
    print("\nResumen de archivos:")
    for archivo, info in resumen_archivos.items():
        estado = "Cumple" if info["cumple"] else "No cumple"
        print(f"{archivo}: {estado}. Registros cargados: {info['registros']}. {info['mensaje']}")

    mycursor.close()
    mydb.close()
    print("\nConexión a la base de datos cerrada.")
else:
    print("No se pudo conectar a la base de datos.")
